# Stage D Quiz - Section 2 (Kaggle Project)

**Build artificial intelligence algorithms to label satellite image chips with different atmospheric conditions and the different classes of land cover/land use.  For this Multi-class Multi-Label problem, some of the labels are from the following categories: Cloud Cover (clear, partly, cloudy, haze), Primary RainForest, Water (rivers, lakes), Habitation (large city, small homes), Agriculture, Roads etc. The algorithms from this project will enable us to understand where, how and why deforestation happens.**

# Import Libraries

In [61]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import numpy as np
from keras import backend as K
from sklearn.metrics import fbeta_score
from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

# Read Dataset

In [62]:
#read dataset
train_data = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')
sample_submission = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Data Analysis and Encoding

In [63]:
#onehot=OneHotEncoder()
encoder = LabelEncoder()

#encoder.fit(train_label['tags'])
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [64]:
#defining a dict of encoded labels
label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [65]:
#loading the traing_images
X = []
Y = []
train_data = shuffle(train_data,random_state=0)

In [66]:
for image_name, tags in tqdm(train_data.values, miniters=400):
    arr = cv2.imread('/kaggle/input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [01:54<00:00, 354.84it/s]


# Data Splitting

In [67]:
X = np.array(X, np.float16)/255.0

In [68]:
#splitting
X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [69]:
del X, Y
gc.collect()

324

In [70]:
gc.collect()

23

In [71]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [72]:
from tensorflow.keras.optimizers import Adam

model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

Epoch 1/45
506/506 [==============================] - 33s 63ms/step - loss: 0.2281 - fbeta: 0.6334 - val_loss: 0.2098 - val_fbeta: 0.6965
Epoch 2/45
506/506 [==============================] - 32s 62ms/step - loss: 0.2002 - fbeta: 0.6815 - val_loss: 0.1944 - val_fbeta: 0.6851
Epoch 3/45
506/506 [==============================] - 32s 63ms/step - loss: 0.1885 - fbeta: 0.7031 - val_loss: 0.1847 - val_fbeta: 0.7327
Epoch 4/45
506/506 [==============================] - 31s 61ms/step - loss: 0.1778 - fbeta: 0.7237 - val_loss: 0.1814 - val_fbeta: 0.7087
Epoch 5/45
506/506 [==============================] - 31s 61ms/step - loss: 0.1693 - fbeta: 0.7397 - val_loss: 0.1676 - val_fbeta: 0.7311
Epoch 6/45
506/506 [==============================] - 31s 61ms/step - loss: 0.1640 - fbeta: 0.7479 - val_loss: 0.1578 - val_fbeta: 0.7561
Epoch 7/45
506/506 [==============================] - 31s 61ms/step - loss: 0.1588 - fbeta: 0.7552 - val_loss: 0.1536 - val_fbeta: 0.7634
Epoch 8/45
506/506 [==============

In [73]:
gc.collect()

1648

In [74]:
#with tpu_strategy.scope():
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 3s 12ms/step - loss: 0.1653 - fbeta: 0.8128
Test loss: 0.1652628481388092
Test accuracy: 0.8128177523612976


In [75]:
gc.collect()

790

In [76]:
# Evaluate the model on the validation set
model.evaluate(x_val, y_val)

# Calculate the f-beta score for the training set
train_fscore = fbeta_score(y_train, np.round(model.predict(x_train)), beta=2,average = 'weighted')
print("train fscore: ", train_fscore)

# Calculate the f-beta score for the validation set
val_fscore = fbeta_score(y_val, np.round(model.predict(x_val)), beta=2, average = 'weighted')
print("val fscore: ", val_fscore)

253/253 [==============================] - 3s 12ms/step - loss: 0.1653 - fbeta: 0.8128
train fscore:  0.8811771588086034
val fscore:  0.8042378359147878


In [77]:
del x_train, y_train, x_val, y_val
gc.collect()

742

In [78]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sample_submission[0 : 40669]
files = sample_submission[40669 : ]

In [79]:
#with tpu_strategy.scope():   
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('/kaggle/input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [00:58<00:00, 351.93it/s]


In [80]:
gc.collect()

69

In [81]:
#with tpu_strategy.scope():
strg = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
strg.append(predictions)

957/957 - 16s


In [82]:
gc.collect()

790

In [83]:
#converting my encoded labels back to it original form
df = np.array(strg[0])
for i in range (1, len(strg)):
    df += np.array(strg[i])
df = pd.DataFrame(df, columns = label_map)

In [84]:
df

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.003660,2.238588e-11,1.562238e-04,1.974943e-02,5.635653e-05,9.999381e-01,3.846910e-07,2.436653e-14,1.927346e-03,2.158016e-03,4.184544e-04,1.238821e-06,0.999998,0.006109,4.402757e-03,5.156556e-10,0.004909
1,0.080094,7.525460e-14,1.423597e-03,6.177402e-02,4.090965e-04,9.995233e-01,6.847761e-11,2.738957e-13,4.028440e-02,3.026277e-03,3.544687e-05,9.802645e-05,1.000000,0.136897,1.608261e-01,8.078378e-08,0.035733
2,0.000444,1.415443e-15,3.288607e-11,1.851315e-08,1.253346e-07,9.113579e-13,1.337442e-10,5.962431e-12,2.215331e-07,5.950621e-10,3.686133e-09,1.000000e+00,1.000000,0.000323,1.575155e-11,5.134713e-16,0.000717
3,0.074279,3.544771e-09,7.995665e-04,1.558542e-02,2.176553e-03,6.694387e-01,3.155474e-09,3.250883e-11,5.763552e-02,6.522298e-03,2.414882e-04,5.573432e-01,1.000000,0.022636,8.185118e-03,3.373260e-07,0.010418
4,0.000103,1.112891e-21,1.044754e-06,1.182667e-11,1.349791e-11,5.767477e-09,3.015366e-08,1.221500e-18,2.108065e-06,6.426093e-06,1.928415e-05,9.999957e-01,1.000000,0.000051,5.381574e-20,1.666527e-16,0.000338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.000067,1.788079e-16,1.914406e-07,1.338591e-17,4.794042e-25,6.738871e-03,9.574043e-01,2.718160e-13,3.793274e-05,2.140000e-01,3.590584e-04,1.772940e-04,0.019262,0.168351,2.943224e-12,1.134209e-13,0.007027
61187,0.137673,4.875626e-06,3.762454e-03,1.952928e-09,2.133894e-18,9.999945e-01,2.502862e-11,4.405455e-07,2.901256e-04,2.660155e-04,1.519570e-06,2.851652e-09,0.983604,0.002865,2.089549e-11,8.694821e-07,0.837821
61188,0.021262,2.968746e-13,3.180590e-05,1.495737e-02,3.659725e-04,9.995973e-01,4.627479e-09,1.865615e-14,1.772836e-02,4.792362e-03,4.566533e-05,8.198278e-05,1.000000,0.014870,5.827755e-03,2.282982e-09,0.010748
61189,0.042246,2.630130e-15,6.501871e-10,1.265242e-12,1.438611e-15,3.104836e-03,8.640648e-01,1.817077e-15,4.125118e-03,1.322925e-04,1.882710e-01,7.739484e-06,0.139216,0.009465,2.259482e-08,2.681127e-14,0.018086


In [87]:
ypreds = []
for i in tqdm(range(df.shape[0]), miniters=1000):
    z = df.loc[[i]]
    z = z.apply(lambda x: x > 0.2, axis=1)
    z = z.transpose()
    z= z.loc[z[i] == True]
    ' '.join(list(z.index))
    ypreds.append(' '.join(list(z.index)))

sample_submission['tags'] = ypreds
sample_submission.to_csv('final_subm.csv', index=False)

100%|██████████| 61191/61191 [02:00<00:00, 507.68it/s]


# Stage D Quiz - Section 1 

Q1. The following are examples of Hyperparameters employed in Neural networks to improve the performance of a model. Choose the option(s) not appropriate
Ans: Momentum
    
Q2. Which of the neural network architectures in the options below perform weight sharing?
Ans: Convolutional Neural Network
    
Q3. Which of the following techniques in a neural network is used to prevent Overfitting?
Ans: All of the above
    
Q4. Which of the following options is TRUE about Perceptrons?
Ans: It is also called a single-layer neural network
    
Q5. Inductive Learning is based on the knowledge that if something happens a lot it is likely to be generally ______
Ans: True

Q6. Which of the following is the structure of the input to an artificial neuron?
Ans: Weighted sum of bias + inputs
    
Q7. Which of the following is FALSE about Deep Learning and Machine Learning algorithm?
Ans: Feature Extraction needs to be done manually in both ML and DL algorithm
    
Q8. Which of the following is a type of Neural Network?
Ans: Convolutional Neural Network (CNN)
    
Q9. What is the purpose of an activation function?
Ans: To normalize the inputs

Q10. What are the factors to consider when selecting the depth of a neural network?
Ans: 1,2,4,5
    
Q11. Which of the following is true for Neural Networks?
I. Artificial neurons are Identical in operations to biological ones. II. Neural networks can be simulated on a conventional computer . III. The training time depends on the size of the network
Ans: II and III
    
Q12. What is back propagation?
Ans: It is the transmission of error back through the network to allow weights to be adjusted so that the network can learn
    
Q13. Which is the most direct application of neural networks?
Ans: Pattern classification

Q14. What are the pros of neural networks over computers?
Ans: All of the above
    
Q15. Which of the following is false?
Ans: None of the above
    
Q16. The fundamental unit of network is
Ans: Neuron
    
Q17. What is the function of the Axon and Dendrite?
Ans: Transmit and Collect signals
    
Q18. Which of the following do you understand is an example of active learning?
I. Automated Vehicles
II. Recommender Systems
III. Dust Cleaning Machine
Ans: II only

Q19. Which of the following is also referred to as explanatory learning?
Ans: Unsupervised learning
    
Q20. How many types of learning do you know are available in machine learning?
Ans: 4